In [4]:
%%writefile ollama_client.py

from ollama import Client
client = Client(
  host='http://localhost:11434',
  headers={'x-some-header': 'some-value'}
)
response = client.chat(model='llama3.2', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])

print(response['message']['content'])

Overwriting ollama_client.py


In [6]:
from ollama import chat

stream = chat(
    model='llama3.2',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. When sunlight enters Earth's atmosphere, it encounters tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. These molecules scatter the light in all directions, but they scatter shorter (blue) wavelengths more than longer (red) wavelengths.
3. This is because the smaller molecules are more effective at scattering the shorter wavelengths.
4. As a result, the blue light is dispersed throughout the atmosphere, reaching our eyes from all directions.
5. Our brains interpret this scattered blue light as the color of the sky.

The reason why the sky isn't always blue is that the amount of sunlight and the angle of the sun can affect the scattering of light. For example:

* During sunrise and sunset, the sun's rays have to travel through more of the atmosphere, which scatters the shorter

### AsyncClient without stream

In [13]:
%%writefile ollama_client_asyncio.py

import asyncio
from ollama import AsyncClient

async def chat():
  message = {'role': 'user', 'content': 'Why is the sky blue?'}
  response = await AsyncClient().chat(model='llama3.2', messages=[message])
  print(response['message']['content'])

asyncio.run(chat())

Overwriting ollama_client_asyncio.py


### AsyncClient with stream=True

In [5]:
%%writefile ollama_client_stream.py

import asyncio
from ollama import AsyncClient

async def chat():
  message = {'role': 'user', 'content': 'Why is the sky blue?'}
  async for part in await AsyncClient().chat(model='llama3.2', messages=[message], stream=True):
    print(part['message']['content'], end='', flush=True)

asyncio.run(chat())

Writing ollama_client_stream.py


In [14]:
%mkdir rag_code_assistant
%cd rag_code_assistant
%mkdir code_snippets

c:\Users\rahul\Documents\Personal_Learning\GenAI LLM\rag_code_assistant


c:\Users\rahul\Documents\Personal_Learning\GenAI LLM\.venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [34]:
%pip install langchain-community langchain-openai langchain-ollama colorama faiss-cpu unstructured

[notice] A new release of pip is available: 24.2 -> 24.3.1

[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ---------- ----------------------------- 262.1/981.5 kB ? eta -:--:--
     -------------------- ----------------- 524.3/981.5 kB 2.4 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ----------- ---------------------------- 0.5/1.7 MB 3.4 MB/s eta 0:00:01
   ----------------------- ---------------- 1.0/1.7 MB 3.1 MB/s eta 0:00:01
   ---------------------------------------- 1.7/1.7 MB 3.3 MB/s eta 0:00:00
Using cached backoff-2.2.1-py3-none-any.whl (15 kB)
   ---------------------------------------- 0.0/586.9 kB ? eta -:--:--
   ---------------------------------------- 586.9/586.9 kB 2.3 MB/s eta 0:00:00
   ------------------------

In [21]:
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


https://dzone.com/articles/enhance-your-workflow-with-ollama-langchain-and-rag

In [24]:
%pip install langchain huggingface_hub sentence_transformers

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ------------------- -------------------- 1.3/2.6 MB 6.1 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 6.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   ---- ----------------------------------- 1.3/11.1 MB 8.4 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.1 MB 7.6 MB/s eta 0:00:02
   ---------------- ----------------------- 4.7/11.1 MB 8.1 MB/s eta 0:00:01
   ------------------------- -------------- 7.1/11.1 MB 8.9 MB/s eta 0:00:01
   --------------------------------- ------ 9.2/11.1 MB 9.2 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 9.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/43.6 MB ? eta -:--:--
   - -------------------------------------- 1.3/43.6 MB 6.7 MB/s eta 0:00:07
   --- ------------------------------------ 3.7/43.6 MB 9.9 MB/s eta 0:00:05
   ----- -------------


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Building the RAG Assistant
Step 1: Create the Index Knowledge Base (index_knowledge_base.py)

In [6]:
%%writefile index_knowledge_base.py

import os
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from colorama import Fore, Style, init

# Initialize colorama for terminal color support
init(autoreset=True)

def create_index():
    openai_api_key = os.environ.get('OPENAI_API_KEY')  # Replace with your actual API key
    try:
        print(f"{Fore.LIGHTBLUE_EX}Loading documents from the 'code_snippets' folder...{Style.RESET_ALL}")
        loader = DirectoryLoader("rag_code_assistant/code_snippets", glob="*.py")
        documents = loader.load()
        print(f"{Fore.LIGHTGREEN_EX}Documents loaded successfully!{Style.RESET_ALL}")

        print(f"{Fore.LIGHTBLUE_EX}Indexing documents...{Style.RESET_ALL}")
        data_store = FAISS.from_documents(documents, OpenAIEmbeddings(openai_api_key=openai_api_key))
        data_store.save_local("index")
        print(f"{Fore.LIGHTGREEN_EX}Knowledge base indexed successfully!{Style.RESET_ALL}")
    except Exception as e:
        print(f"{Fore.RED}Error in creating index: {e}{Style.RESET_ALL}")



Overwriting index_knowledge_base.py


In [7]:
!python index_knowledge_base.py

Step 2: Create the Retrieval Pipeline (retrieval_pipeline.py)

In [3]:
%%writefile retrieval_pipeline.py

from langchain_community.vectorstores import FAISS
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_ollama.llms import OllamaLLM
from colorama import Fore, Style, init
import asyncio

# Initialize colorama for terminal color support
init(autoreset=True)

openai_api_key = os.environ.get('OPENAI_API_KEY')  # Replace with your actual API key

try:
    print(f"{Fore.LIGHTYELLOW_EX}Loading FAISS index...{Style.RESET_ALL}")
    data_store = FAISS.load_local(
        "index",
        OpenAIEmbeddings(openai_api_key=openai_api_key),
        allow_dangerous_deserialization=True
    )
    print(f"{Fore.LIGHTGREEN_EX}FAISS index loaded successfully!{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}Error loading FAISS index: {e}{Style.RESET_ALL}")

try:
    print(f"{Fore.LIGHTYELLOW_EX}Initializing the Ollama LLM...{Style.RESET_ALL}")
    llm = OllamaLLM(model="llama3.2", base_url="http://localhost:11434", timeout=120)
    print(f"{Fore.LIGHTGREEN_EX}Ollama LLM initialized successfully!{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}Error initializing Ollama LLM: {e}{Style.RESET_ALL}")

def query_pipeline(query):
    try:
        print(f"{Fore.LIGHTBLUE_EX}Searching the knowledge base...{Style.RESET_ALL}")
        context = data_store.similarity_search(query)
        print(f"{Fore.LIGHTGREEN_EX}Context found:{Style.RESET_ALL} {context}")

        prompt = f"Context: {context}\nQuestion: {query}. Response only if context is relevant to calculator code, else response sorry cannot help you in this context."
        print(f"{Fore.LIGHTBLUE_EX}Sending prompt to LLM...{Style.RESET_ALL}")
        response = llm.generate(prompts=[prompt])
        print(f"{Fore.LIGHTGREEN_EX}Received response from LLM.{Style.RESET_ALL}")
        return response
    except Exception as e:
        print(f"{Fore.RED}Error in query pipeline: {e}{Style.RESET_ALL}")
        return f"{Fore.RED}Unable to process the query. Please try again.{Style.RESET_ALL}"

Overwriting retrieval_pipeline.py


Step 3: Create the Interactive Assistant (interactive_assistant.py)

In [4]:
%%writefile interactive_assistant.py

from retrieval_pipeline import query_pipeline
from colorama import Fore, Style, init

# Initialize colorama for terminal color support
init(autoreset=True)

def interactive_mode():
    print(f"{Fore.LIGHTBLUE_EX}Start querying your RAG assistant. Type 'exit' to quit.{Style.RESET_ALL}")
    while True:
        try:
            query = input(f"{Fore.LIGHTGREEN_EX}Query: {Style.RESET_ALL}")

            if query.lower() == "exit":
                print(f"{Fore.LIGHTBLUE_EX}Exiting... Goodbye!{Style.RESET_ALL}")
                break

            print(f"{Fore.LIGHTYELLOW_EX}Processing your query...{Style.RESET_ALL}")
            response = query_pipeline(query)
            print(f"{Fore.LIGHTCYAN_EX}Assistant Response: {Style.RESET_ALL}{response.generations[0][0].text}")

        except Exception as e:
            print(f"{Fore.RED}Error in interactive mode: {e}{Style.RESET_ALL}")

Overwriting interactive_assistant.py


Step 4: Create the Main Script (main.py)


In [5]:
%%writefile main.py 

from interactive_assistant import interactive_mode
from index_knowledge_base import create_index
from colorama import Fore, Style, init

# Initialize colorama for terminal color support
init(autoreset=True)

print(f"{Fore.LIGHTBLUE_EX}Initializing the RAG-powered Code Assistant...{Style.RESET_ALL}")

try:
    print(f"{Fore.LIGHTYELLOW_EX}Indexing the knowledge base...{Style.RESET_ALL}")
    create_index()
    print(f"{Fore.LIGHTGREEN_EX}Knowledge base indexed successfully!{Style.RESET_ALL}")
except Exception as e:
    print(f"{Fore.RED}Error while indexing the knowledge base: {e}{Style.RESET_ALL}")

try:
    print(f"{Fore.LIGHTYELLOW_EX}Starting the interactive assistant...{Style.RESET_ALL}")
    interactive_mode()
except Exception as e:
    print(f"{Fore.RED}Error in interactive assistant: {e}{Style.RESET_ALL}")

Overwriting main.py
